# <h1>MDP RESOLUTION FOR THE RADIO RESOURCE ALLOCATION PROBLEM</h1>

<ul>
    <li><a href="routines definition">The Routines Definition</a> section should be ran first for initialization.</li>
    <li><a href="Tests">The Tests</a> section is for verifying the various modules.</li>
    <li><a href="Simulations">The Simulations</a> section is to compare the abstraction models and try the selected one.</li>
</ul>

## <h2 id="routines definition">ROUTINES DEFINITION</h2>

In [2]:
import time
import os
import shutil
import csv
import torch
import numpy as np
import math
import itertools
from itertools import product
import torch.nn.functional as F
import pandas as pd
from scipy.stats import poisson
import warnings
warnings.filterwarnings("ignore")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("CUDA is AVAILABLE" if torch.cuda.is_available() else "cuda is NOT available")

CUDA is AVAILABLE


In [3]:
def soft_shift_poisson(rate:int, left_shift:int=0) -> float:
    """
    Returns the mathematical expectation of max(0,X-left_shift), where X a Poisson distribution.

    Args:
        rate (int).
        left_shift (int, optional). Defaults to 0.
    """
    if left_shift == 0: return rate
    sub_probability = poisson.cdf(2*left_shift-1,rate)
    sub_mean =  0
    for n in range(2*left_shift): sub_mean += n*poisson.pmf(n,rate)
    remaining_mean = rate - sub_mean
    remaining_probability = 1 - sub_probability
    return remaining_mean - remaining_probability*left_shift

def small_MDP_solution(num_states:int, num_actions:int, transition_matrix:torch.tensor, cost_matrix:torch.tensor, discount_factor:float=0.9, precision:float=1e-16) -> torch.tensor:
    """Solution of an MDP (S, A, T, C) with small state and action spaces.

    Args:
        num_states (int): Number of states. Assume S = {0, ..., num_states-1}.
        num_actions (int): Number of actions. Assume A = {0, ..., num_actions-1}.
        transition_matrix (torch.tensor): Tensor representing the transition matrix T. Assume T[a,s,s'] is the probability that the system transitions to state s' when action a was taken in state s.
        cost_matrix (torch.tensor): Tensor representing the cost matrix C. Assume C[a,s] is the expected cost for taking action a in state s.
        discount_factor (float, optional). Defaults to 0.9.
        precision (float, optional): Upper bound of the distance (in norm \infty) between the optimal value function and the computed value function. Defaults to 1e-16.

    Returns:
        torch.tensor: (2, num_states) matrix. Each column corresponds to a state. The first line is the action suggested, the second line is the value estimated.
    """
    precision = torch.tensor([precision],dtype=float).to(device).item()
    transition_matrix, cost_matrix = transition_matrix.to(device).to(float), cost_matrix.to(device)
    discount_factor = torch.tensor([discount_factor],dtype=float).to(device)
    error_factor = discount_factor / (one-discount_factor)
    states_vector = torch.tensor(range(num_states),dtype=int).to(device)
    actions_vector = torch.tensor(range(num_actions),dtype=int).to(device)
    old_value, new_value = torch.zeros([num_states],dtype=float).to(device), torch.zeros([num_states],dtype=float).to(device)
    error = "#NA"
    policy = torch.zeros([num_states],dtype=int).to(device)
    while True:
        for s in states_vector:
            s_p = s.item() 
            print(f"Resolution of the small MDP.   Current error = {error};  current state number = {s_p}", end="\r")
            Q = torch.zeros([num_actions],dtype=float).to(device)
            for a in actions_vector: Q[a] = cost_matrix[a,s] + discount_factor*torch.dot(transition_matrix[a,s],old_value)
            new_value[s] = torch.min(Q)
            policy[s] = torch.argmin(Q)
        diff_max, diff_min = torch.max(new_value - old_value), torch.min(new_value - old_value)
        diff_scope = diff_max - diff_min
        error = (error_factor * diff_scope).item()
        # error = torch.max(torch.abs(new_value-old_value))
        if error <= precision: break
        old_value = new_value.clone()
    print(" "*150, end="\r")
    return torch.stack([policy, new_value])

# Place the more usual numbers on the device.
two = torch.tensor([2],dtype=float).to(device)
one = torch.tensor([1],dtype=float).to(device)
zero = torch.tensor([0],dtype=float).to(device)

In [4]:
class ground_model:
    """
    Definition and operations for a radio resource allocation problem.
    Parameters:
    max_size = maximum number of bits in an user equipment's queue;
    number of user equipments; number of resource blocks;
    arrival_rates_set_to_default: indicate if the arrival rate at each user equipment's buffer is set to default
    CQIs_are_equal: indicates if the chanel quality index is constant in the UE and in the RB;
    CQI_base = number of bits scheduled for transmission when CQI=1;
    coef_of_drop, coef_of_latency, power_of_drop, power_of_latency = α, β, x, y:
    cost = α*(cost for rejection)**x + β(cost for latency)**y;
    discount factor;
    precision = bound of the difference between the calculated and the optimal value function of the MDP;
    path = location of the results of the operations (if it is not indicated from the root, the location is in the current directory).
    """
    
    def __init__(self, max_size:int=3, number_UEs:int=6, number_RBs:int=2, 
                 arrival_rates_set_to_default=None,
                 CQIs_are_equal=True, CQI_base:float=1,
                 coef_of_drop:float=1, coef_of_latency:float=1, power_of_drop:float=1, power_of_latency:float=1, 
                 discount_factor:float=0.9, precision:float=1e-16,
                 path=None) -> None:
        
        # States and actions
        number_states, number_actions = (max_size + 1) ** number_UEs, number_UEs ** number_RBs
        range_UE_indices = torch.tensor(range(number_UEs))
        range_number_bits = torch.tensor(range(max_size+1))
        list_states = list(product(range_number_bits,repeat=number_UEs))
        list_actions = list(product(range_UE_indices, repeat=number_RBs))
        
        # Send to the device: the data, the ranges for indices, the states and the actions
        self.max_size = torch.tensor([max_size],dtype=int).to(device)
        self.number_UEs, self.number_RBs = torch.tensor([number_UEs],dtype=int).to(device), torch.tensor([number_RBs],dtype=int).to(device)
        self.coef_of_drop, self.coef_of_latency = torch.tensor([coef_of_drop],dtype=float).to(device), torch.tensor([coef_of_latency],dtype=float).to(device)
        self.power_of_drop, self.power_of_latency = torch.tensor([power_of_drop],dtype=float).to(device), torch.tensor([power_of_latency],dtype=float).to(device)
        self.discount_factor, self.precision = torch.tensor([discount_factor], dtype=float).to(device), torch.tensor([precision], dtype=float).to(device)
        self.number_states, self.number_actions = torch.tensor([number_states]).to(device), torch.tensor([number_actions]).to(device)
        self.range_UE_indices, self.range_RB_indices = torch.tensor(range(number_UEs)).to(device), torch.tensor(range(number_RBs)).to(device)
        self.range_state_indices, self.range_action_indices = torch.tensor(range(number_states)).to(device), torch.tensor(range(number_actions)).to(device)
        self.states_matrix, self.actions_matrix = torch.tensor(list_states).to(device), torch.tensor(list_actions).to(device)
        self.empty_buffer = torch.zeros([self.number_UEs],dtype=int).to(device)                      # Number of bits in each UE's empty buffer
        self.full_buffer = self.max_size*torch.ones([self.number_UEs],dtype=int).to(device)          # Number of bits in each UE's full buffer
        
        # Get the arrival rates
        if arrival_rates_set_to_default:
            arrival_rates_vector = torch.ones([self.number_UEs]).to(device)
            fraction = torch.tensor([max_size/3],dtype=float).to(device)
            print(f"Arrival rate is set to default for each UE: lambda = {fraction.item()}")
            arrival_rates_vector = fraction*arrival_rates_vector
        else:
            arrival_rates_vector = torch.ones([self.number_UEs]).to(device)
            print("Setting the arrival rate for each UE.")
            decision = input("Same arrival rate? (y/n)")
            if "Y" in decision.upper():
                value_ = float(input("For all UE, arrival rate is: "))
                value_ = torch.tensor([value_],dtype=float).to(device)
                arrival_rates_vector = value_*arrival_rates_vector
            else:
                for i in self.range_UE_indices:
                    print(f"Arrival rate for UE {i}: ")
                    arrival_rates_vector[i] = float(input())
        self.arrival_rates_vector = arrival_rates_vector
        
        # Get the chanel quality indices (CQIs) matrix
        if CQIs_are_equal:
            self.CQI_matrix = CQI_base*torch.ones([self.number_UEs,number_RBs],dtype=float).to(device)
        else:
            self.CQI_matrix = torch.ones([self.number_UEs,number_RBs],dtype=float).to(device)
            print("Definition of the chanel quality indices.")
            decision = input("Allow various values of CQI? (y/n): ")
            if "Y" in decision.upper():
                print("\nFor each UE and each RB, let's define the CQI as:  CQI = (coef of UE and RB)*(common base)")
                print("Enter all the coefficients. Later on, you will enter the common base to multiply.")
                for i,j in product(self.range_UE_indices,self.range_RB_indices):
                    print(f"    Coefficient for UE {i} and RB {j}: ", end="")
                    time.sleep(0.4)
                    value_ = input()
                    if value_.isnumeric(): 
                        print(value_)
                        value_ = torch.tensor([float(value_)],dtype=float).to(device)
                        self.CQI_matrix[i,j] = value_
                value_ = input("Enter the common base: ")
                if value_.isnumeric(): 
                    value_ = torch.tensor([float(value_)],dtype=float).to(device)
                    CQI_base = value_
                self.CQI_matrix = CQI_base*self.CQI_matrix
                # self.CQI_matrix = self.CQI_matrix.to(device)
                
        # Prepare the resolution
        self.prepare_resolution()
        
    def prepare_resolution(self, directory_root=None, solution=None, from_scratch = True):
        
        # Prepare the path for the solution and the simulations
        if from_scratch:
            if directory_root == None:
                this_directory = os.getcwd()
                solution_directory = "Solution_for_B-" + str(self.max_size.item()) + "_UE-" + str(self.number_UEs.item()) + "_RB-" + str(self.number_RBs.item())
                directory_root = os.path.join(this_directory, solution_directory)
            last_index = 1
            while True:
                directory = (directory_root if last_index==1 else directory_root + "_"+str(last_index))
                if os.path.exists(directory):
                    print(f"Directory {directory} serves for the resolution.")
                    decision = input("Continue the last resolution in it? (Y/N): ")
                    if "Y" in decision.upper(): break
                    else:
                        print(f"This is, you should decide wether delete the existing resolution in {directory}")
                        print("and start a new resolution in this directory or let me look for another directory.")
                        decision = input("Start a new resolution in it? (Y/N): ")
                        if "Y" in decision.upper():
                            shutil.rmtree(directory)
                            os.makedirs(directory)
                            break
                        else: last_index += 1
                else:
                    os.makedirs(directory)
                    break
            self.directory = directory
        
        # Prepare the initial value for the backup iteration and save it in a dedicated file
        solution_path = os.path.join(self.directory, "ground_solution.pth")
        if os.path.exists(solution_path):
            if solution == None: self.solution = torch.load(solution_path).to(device)
            else:
                self.solution = solution
                torch.save(self.solution, solution_path)
        else:
            self.solution = (torch.zeros([2,self.number_states],dtype=float).to(device) if solution==None else solution)
            torch.save(self.solution, solution_path)
        self.solution_path = solution_path
        
        # Prepare the file to store the error of running the backup iteration
        self.error_path = error_path = os.path.join(self.directory, "ground_error.txt")
        if os.path.exists(error_path):
            with open(error_path, "r") as f:
                error = f.read()
                self.error = (float(error) if error.isnumeric() else "#NA")
        else:
            with open(error_path, "w") as f:
                self.error = "#NA"
                f.write(self.error)
        
    
    def remainder_fn(self, state:torch.tensor, action:torch.tensor) -> torch.tensor:
        """
        Returns the number of bits remaining in each UE's buffer if action of index action_index was taken in state of index state_index.
        """
        state = state.to(device)
        schedule = torch.zeros([self.number_UEs],dtype=float).to(device)  # Number of bits scheduled for transmission
        for i,j in product(self.range_UE_indices,self.range_RB_indices): 
            if action[j]==i:
                schedule[i] += self.CQI_matrix[i,j]
        schedule = schedule.to(int)
        angry_remainder = state - schedule    # Number of remaining bits if negative values are possible
        return torch.max(self.empty_buffer, angry_remainder)
    
    def transition_probability_fn(self, state1:torch.tensor, action:torch.tensor, state2:torch.tensor) -> torch.tensor:
        def UE_transition_probability_fn(rest:int, rate:float, next:int) -> torch.tensor:
            max_size = self.max_size
            sure, impossible = torch.tensor([1],dtype=float).to(device), torch.tensor([0],dtype=float).to(device)
            if next<rest:     return impossible                                                           # next < rest
            if next<max_size: return torch.tensor([poisson.pmf(next.item()-rest.item(),rate.item())],dtype=float).to(device)   # rest ≤ next < max_size  
            range_next = torch.tensor(range((max_size-rest).item()),dtype=int).to(device)                 # next = max_size, then  P{size=next} = 1 - P{arrival<}
            probability = sure
            for arrival in range_next:
                probability_arrival = torch.tensor([poisson.pmf(arrival.item(),rate.item())],dtype=float).to(device)
                probability = probability - probability_arrival
            return probability
        rate = self.arrival_rates_vector
        remainder = self.remainder_fn(state1,action)
        probability = torch.tensor([1],dtype=float).to(device)
        for i in self.range_UE_indices:                                                                    # i = queue index
            probability = probability * UE_transition_probability_fn(remainder[i], rate[i], state2[i])
        return probability
    
    def cost_fn(self, state:torch.tensor, action:torch.tensor) -> torch.tensor:
        def exact_cost_fn(remainder:torch.tensor, state2:torch.tensor) -> torch.tensor:                     # Exact cost associated with the transition from state1 to state2
            def partial_cost_fn(rest:torch.tensor, arrival:torch.tensor) -> torch.tensor:                   # Cost for transition of the queue
                false_excess = arrival + rest - self.max_size
                excess = torch.max(nothing, false_excess)
                return self.coef_of_drop * excess**self.power_of_drop + self.coef_of_latency * rest**self.power_of_latency
            
            exact_cost = torch.tensor([0],dtype=float).to(device)
            for q in self.range_UE_indices:
                if state2[q] < remainder[q]:                                                                 # The transition is impossible
                    exact_cost = torch.tensor([0],dtype=float).to(device)
                    break
                else:
                    arrival = state2[q] - remainder[q]
                    exact_cost = exact_cost + partial_cost_fn(remainder[q], arrival)
            return exact_cost
        
        nothing = torch.tensor([0],dtype=int).to(device)
        remainder = self.remainder_fn(state, action)
        cost = torch.tensor([0],dtype=float).to(device)
        for s in self.range_state_indices:
            possible_state = self.states_matrix[s]
            possible_exact_cost = exact_cost_fn(remainder, possible_state)
            cost = cost + possible_exact_cost * self.transition_probability_fn(state,action,possible_state)
        return cost
    
    def resolution(self, directory=None, solution=None, precision=None) -> torch.tensor:
        
        # Initialization
        if directory == None: 
            directory = self.directory
            from_scratch = False
        else:
            from_scratch = True
        if precision == None: precision = self.precision
        self.prepare_resolution(directory, solution, from_scratch)
        
        # Body of the resolution
        gamma = self.discount_factor.item()
        gamma_prime = gamma / (1-gamma)
        old_value = self.solution[1].to(device)
        new_value = self.solution[1].to(device)
        current_policy, current_value = self.solution[0].to(device), self.solution[1].to(device)
        policy = torch.zeros([self.number_states],dtype=int).to(device)
        error = self.error
        step = 0
        try:
            while True:
                step += 1
                for s in self.range_state_indices:
                    print(f"Ground model resolution.   Updating the value (step {step}).  Current error: {error};  current state: s_{s}", end="\r")
                    state1 = self.states_matrix[s]
                    Q = torch.zeros([self.number_actions],dtype=float).to(device)
                    for a in self.range_action_indices:
                        action = self.actions_matrix[a]
                        P = torch.zeros([self.number_states],dtype=float).to(device)
                        for s2 in self.range_state_indices: 
                            state2 = self.states_matrix[s2]
                            P[s2] = self.transition_probability_fn(state1, action, state2)
                        Q[a] = self.cost_fn(state1,action) + self.discount_factor*torch.sum(old_value*P)
                    new_value[s] = torch.min(Q)
                    policy[s] = torch.argmin(Q)
                current_policy, current_value = policy.clone(), new_value.clone()
                diff = torch.abs(new_value - old_value)
                max_diff = torch.max(diff).item() 
                error = gamma_prime * max_diff
                if error <= precision: break
        except KeyboardInterrupt:
            pass 
        print(f"Ground model resolution.   Updating the value (step {step}).  Current error: {error};  current state: s_{s}", end="\r")
        
        # Storing the solution and the error
        self.solution = torch.stack([current_policy, current_value])
        torch.save(self.solution, self.solution_path)
        with open(self.error_path, "w") as f:
            f.write(str(error))

In [5]:
class abstract_model:
    """Make and solve an abstract model corresponding to a ground model

    Args:
        model (ground_model): the ground model on which the abstract model is built.
        number_groups (int, optional): number of groups of UEs fo the abstract model. Defaults to 3.
        coef_owners (str, optional): the object that receive a weight: "UEs" or "groups". Defaults to "UEs".
        coef_distribution_criterion (str, optional): criterion to weight these objects: 
        "uniform" weighting, by "similarity" or by "dissimilarity". Defaults to "uniform".
        variant (str, optional): variant for the coefficient distribution criterion: "standard deviation" (or "sd"),
        "cross entropy" (or "cross") or "total difference" (or "Gini index" or "Gini"). Necessary for groups dissimilarity. Defaults to "sd".        
        selection_mode (str, optional): number of representatives in each class: "one", the top weighted ("top") or "all". Defaults to "one".
    """
    
    def __init__(self, model:ground_model, number_groups:int=3, coef_owners="UEs", coef_distribution_criterion="uniform", variant=None, selection_mode="one"): 
        
        # Get ready to use the entered parameters
        if coef_owners == None or coef_owners.upper() in ["UES", "UE"] : coef_owners = "UEs"
        elif coef_owners.upper() in ["GROUP", "GROUPS"]: coef_owners = "groups"
        else:
            print(f'The definition of the abstract model is broken. Cause: objects to weights "{coef_owners}"' + 
                  'has unexpected value. Correct values: "UEs" and "groups".')
            return
        
        if coef_distribution_criterion == None or "UNIF" in coef_distribution_criterion.upper(): 
            coef_distribution_criterion = "uniform"
        elif "DIS" in coef_distribution_criterion.upper(): coef_distribution_criterion = "dissimilarity"
        elif "SIM" in coef_distribution_criterion.upper(): coef_distribution_criterion = "similarity"
        else:
            print(f'The definition of the abstract model is broken. Cause: weighting criterion "{coef_distribution_criterion}' + 
                  'has unexpected value. Correct values: "uniform", "similarity" and "dissimilarity".')
            return
        
        if selection_mode == None or selection_mode == 1 or selection_mode.upper() in ["1", "ONE"]: selection_mode = "one"
        elif "TOP" in selection_mode.upper() or "MOST" in selection_mode.upper(): selection_mode = "top"
        elif selection_mode.upper() in ["ALL", "AL"]: selection_mode = "all"
        else:
            print(f'The definition of the abstract model is broken. Cause: selection mode "{selection_mode}' + 
                  'has unexpected value. Correct values: "one", "top" and "all".')
            return
        
        if coef_owners == "groups" and coef_distribution_criterion == "dissimilarity":
            if variant == None or "ST" in variant.upper() or "SD" in variant.upper(): variant = "sd"
            elif "CROS" in variant.upper(): variant = "cross"
            elif "GINI" in variant.upper() or "TOTAL" in variant.upper(): variant = "gini"
            else:
                print(f'The definition of the abstract model is broken. Cause: variant "{coef_distribution_criterion}' + 
                    'has unexpected value. Correct values for group dissimilarity: "standard deviation", "cross entropy" and "Gini index".')
                return
        else: variant = None
        
        self.model = model
        self.number_groups = torch.tensor(number_groups,dtype=int).to(device)
        self.coef_owners, self.coef_distribution_criterion, self.selection_mode = coef_owners, coef_distribution_criterion, selection_mode
        self.variant = variant
        
        abstraction_beginning_time = time.time()
        self.make_groups_UEs()  # Group the UEs in groups. Result: group_indices_vector and groups_list, number of UEs per group
        self.make_abstraction_function()  # Make the abstract states, the vector of indices of class corresponding to each state, and the list of classes (each class is a tensor)
        self.make_weights_distribution()  # Built the vector weight_distribution representing the weight distribution
        self.make_abstract_problem()  # Build the transition and cost matrices transition_matrix and cost_matrix for the abstract problem
        abstraction_end_time = time.time()
        self.abstraction_elapse_time = abstraction_end_time - abstraction_beginning_time
        
        
    def make_groups_UEs(self) -> torch.tensor:  # Group the UEs in groups
        def grouping(N):
            model = self.model
            characteristics_vector = model.number_UEs/model.number_RBs*torch.sum(model.CQI_matrix,dim=1) - model.arrival_rates_vector
            if N >= model.number_UEs:
                if characteristics_vector.unique().size(0) == characteristics_vector.size(0):
                    sorted_indices = torch.argsort(characteristics_vector)
                    num_elements = characteristics_vector.size(0)
                    vector_groups = torch.zeros(num_elements, dtype=torch.long)
                    vector_groups[sorted_indices] = torch.arange(num_elements)
                    return vector_groups
                else:
                    return grouping(model.number_UEs.item() - 1)
            quantiles = torch.linspace(0, 1, N+1)[1:-1].to(float).to(device)
            bounds_characteristics = torch.quantile(characteristics_vector, quantiles)
            _, group_indices_vector = torch.searchsorted(bounds_characteristics, characteristics_vector.unsqueeze(1), right=True).squeeze(1).unique(return_inverse=True)
            return group_indices_vector
        
        self.group_indices_vector = grouping(self.number_groups)
        self.number_groups = self.group_indices_vector.unique().size(0)
        self.range_group_indices = torch.tensor(range(self.number_groups),dtype=int)
        self.groups_list = [torch.where(self.group_indices_vector==g)[0] for g in self.range_group_indices]
        groups_size_list = [self.groups_list[g].size(0) for g in self.range_group_indices]
        self.groups_size_vector = torch.tensor(groups_size_list,dtype=int).to(device)
    
    def make_abstraction_function(self):  # Make the matrix of abstract states (states_matrix), the vector of indices of class corresponding to each state(states_vector), 
        # and the list of classes (classes_list: each class is a tensor)
        model = self.model
        
        # Build the abstract states
        max_size_groups = torch.zeros([self.number_groups],dtype=int).to(device)
        for g in self.range_group_indices: max_size_groups[g] = model.max_size * self.groups_list[g].size(0)
        states = itertools.product(*[range(size+1) for size in max_size_groups])
        self.states_matrix = torch.tensor(list(states)).to(device)
        self.number_states = self.states_matrix.size(0)
        
        # Build the abstract function as a vector called states_vector. states_vector[gs_idx] is the index 
        # of the abstract state corresponding to ground state of index gs_idx
        self.states_vector = torch.zeros([model.number_states],dtype=int)
        for gs_idx in model.range_state_indices:
            print(f"Getting the abstraction function.  Ground state {gs_idx+2} " + 
                  f"/ {model.number_states.item()+1}  ({(gs_idx+2)/(model.number_states+1).item():.0%})", end="\r")
            ground_state = model.states_matrix[gs_idx]
            abstract_state = torch.zeros([self.number_groups],dtype=int).to(device)
            for g in self.range_group_indices:
                abstract_state[g] = ground_state[self.groups_list[g]].sum()
            for as_idx, row in enumerate(self.states_matrix):
                if torch.all(row == abstract_state):
                    self.states_vector[gs_idx] = as_idx
                    break
        print(" "*100+"\r",end="\r")
        
        # Build the list of classes. classes_list[c_idx] is the class (vector of ground state indices) of ground states 
        # whom abstract state is of index s_idx
        self.range_class_indices = torch.tensor(range(self.number_states),dtype=int)
        self.classes_list = [torch.where(self.states_vector==c_idx)[0] for c_idx in self.range_class_indices]
    
    
    def make_weights_distribution(self):  # Built the vector representing the weight distribution
        def selection_fn(weight_distribution:torch.tensor) -> torch.tensor:  # Return a final weight distribution 
            # corresponding to the selection mode
            if self.selection_mode == "all": return weight_distribution
            m = self.model
            final_weights_vector = torch.zeros(m.number_states,dtype=float).to(device)
            for c_idx in range(len(self.classes_list)): 
                states = self.classes_list[c_idx].to(device)  # Indices of the states of the current class
                local_weights = weight_distribution[states]  # Vector of weights of these states
                max_weight = local_weights.max()  # Maximum value of the local weights
                top_states_positions = torch.nonzero(local_weights==max_weight).squeeze()  # Local positions within the current class of the states with maximum weight value
                if top_states_positions.dim()==0:  # If exactly one state in the class has the local maximum weight value
                    for s_idx in states:
                        final_weights_vector[s_idx] = (1 if weight_distribution[s_idx]==max_weight else 0)
                else:
                    top_states = states[top_states_positions]  # Global indices of the states with the maximum local weight value
                    num_top_states = top_states.size(0)
                    if self.selection_mode == "top":
                        for s_idx in states: 
                            final_weights_vector[s_idx] = (1/num_top_states if s_idx in top_states else 0)
                    else:
                        random_position_in_tops = torch.randint(0, num_top_states, (1,))
                        random_top_idx = top_states[random_position_in_tops]
                        for s_idx in states:
                            final_weights_vector[s_idx] = (1 if random_top_idx==s_idx else 0)
                states_and_weights = torch.stack((states, final_weights_vector[states]), dim=0)
                # print(f"c_{c_idx} ≡ {self.states_matrix[c_idx].cpu().numpy()}\n{states_and_weights.cpu().numpy()}\n")
            return final_weights_vector
        
        def weight_fn(distribution:torch.tensor) -> torch.tensor:  # Return a weight distribution any distribution of positive numbers on the states
            m = self.model
            state_weights_vector = torch.zeros(m.number_states,dtype=float).to(device)
            for c_idx in range(len(self.classes_list)): 
                class_states = self.classes_list[c_idx]
                class_weight = distribution[class_states].sum()
                if class_weight == 0:
                    for s_idx in class_states: state_weights_vector[s_idx] = 1
                else:
                    for s_idx in class_states: state_weights_vector[s_idx] = distribution[s_idx]/class_weight
            return state_weights_vector
        
        def preweight_fn() -> torch.tensor:  # Distribute numbers on states according to the weights of UEs or groups
            m = self.model
            state_weights_vector = torch.zeros(m.number_states,dtype=float).to(device)
            for s_idx in m.range_state_indices: 
                coefs_vector = coef_fn(s_idx)
                state_weights_vector[s_idx] = coefs_vector.sum()
            return state_weights_vector
        
        def coef_fn(state_idx:int) -> torch.tensor:  # Make a coefficient distribution on UEs or groups according to the state of index state_idx
            state = self.model.states_matrix[state_idx]
            criterion, coef_owners, variant = self.coef_distribution_criterion, self.coef_owners, self.variant
            
            # The case of uniform distribution is the most simple: all queues or groups have the same coefficient.
            if criterion == "uniform":
                return (torch.ones([self.model.number_UEs],dtype=float).to(device) 
                        if coef_owners=="UEs" 
                        else torch.ones([self.number_groups],dtype=float).to(device))  # else means "groups"
            
            # From this point on, the criterion is either "similarity" or "dissimilarity".
            abstract_state_idx = self.states_vector[state_idx]  # Everything relies on the current abstract state.
            abstract_state = self.states_matrix[abstract_state_idx]
            
            # The criterion is either "similarity" or "dissimilarity", and the owners of the weights are the UEs:
            if coef_owners == "UEs":
                group_means_vector = abstract_state / self.groups_size_vector  # Average number of bits in each group
                deviations_vector = torch.zeros([self.model.number_UEs],dtype=float).to(device)  # Difference between the number of bits in each buffer and the corresponding group. Init value.
                for ue in self.model.range_UE_indices:
                    g_idx = self.group_indices_vector[ue]  # Group of the UE of index ue
                    deviations_vector[ue] = torch.abs(state[ue] - group_means_vector[g_idx])
                return (torch.exp(-deviations_vector) if criterion=="similarity" else deviations_vector)  # else means "dissimilarity"
            
            # From this point on, the criterion is either "similarity" or "dissimilarity", and the owners of the weights are the groups
            coefs_vector = torch.zeros([self.number_groups],dtype=float).to(device)  # For the vector of coefficients to return
            if criterion == "similarity":  # The criterion is "similarity" and the owners of the weights are the groups
                for g_idx in self.range_group_indices:
                    UEs_vector = torch.where(self.group_indices_vector==g_idx)[0]
                    state_restriction = state[UEs_vector]  # Sub-vector of the ground state corresponding to the ground state of index g_Idx
                    num_UEs = self.groups_size_vector[g_idx]  # Number of UE buffers in this group
                    num_bits = abstract_state[g_idx]  # Total number of bits in this group
                    redistribution = torch.full((num_UEs,),num_bits).to(device)  # Assign this total to each UE. The average sounds better, but the total is easier and returns the same cosine
                    coefs_vector[g_idx] = F.cosine_similarity(state_restriction.to(float),redistribution,dim=0)
                return coefs_vector
            
            # From this point on, the criterion is "dissimilarity", and the owners of the weights are the groups
            if variant == "sd":  # The criterion is "dissimilarity", and the groups should be weighted according to the standard deviation of the bits distribution in each of them
                for g_idx in self.range_group_indices:
                    num_bits = abstract_state[g_idx]  # Total number of bits in this group
                    if num_bits == 0: coefs_vector[g_idx] = 1
                    else:
                        num_UEs = self.groups_size_vector[g_idx]  # Number of UE buffers in this group
                        average_num_bits = num_bits / num_UEs
                        redistribution = torch.full((num_UEs,),average_num_bits).to(device)  # Redistribute the bits of the group equally to its UE-members
                        UEs_vector = torch.where(self.group_indices_vector==g_idx)[0]
                        state_restriction = state[UEs_vector]  # Sub-vector of the ground state corresponding to the ground state of index g_Idx
                        square_deviations_vector = (state_restriction - redistribution)**2
                        variance = math.sqrt(torch.sum(square_deviations_vector))  # Variance of the bits distribution in the group
                        coefs_vector[g_idx] = variance / num_bits  # Coefficient of the group
                return coefs_vector
            
            # From this point on, the criterion is "dissimilarity", and the groups should be weighted according to either the cross entropy or the Gini coefficient of the bits distribution in each of them
            if variant == "cross":  # The criterion is "dissimilarity", and the groups should be weighted according to the cross entropy of the bits distribution in each of them
                for g_idx in self.range_group_indices:
                    # states_idx_vector = torch.where(self.group_indices_vector==g_idx)[0]
                    group_size = self.groups_size_vector[g_idx].item()
                    num_bits = abstract_state[g_idx]
                    entropy = math.log(group_size)*num_bits
                    coefs_vector[g_idx] = entropy
                return coefs_vector
            
            # Here, the criterion is "dissimilarity", and the groups should be weighted according the Gini coefficient of the bits distribution in each of them
            for g_idx in self.range_group_indices:
                if abstract_state[g_idx] != 0:
                    UEs_vector = torch.where(self.group_indices_vector==g_idx)[0]  # UEs in the current group
                    sum_dif = torch.tensor([0],dtype=float).to(device)
                    for i, j in itertools.product(UEs_vector, repeat=2):
                        sum_dif = sum_dif + torch.abs(state[i] - state[j])
                    group_size = self.groups_size_vector[g_idx]
                    num_bits = abstract_state[g_idx]
                    coefs_vector[g_idx] = sum_dif / (two*group_size*num_bits)
            return coefs_vector
        preweights_distribution = preweight_fn()
        weight_distribution = weight_fn(preweights_distribution)
        self.weight_distribution = selection_fn(weight_distribution)
    
    def make_abstract_problem(self) -> None:
        m = self.model
        NA, NS = m.number_actions, self.number_states
        num_steps = NA*NS*NS
        transition_matrix = torch.zeros([NA,NS,NS],dtype=float).to(device)
        cost_matrix = torch.zeros([NA,NS],dtype=float).to(device)
        step = zero
        variant = ("--" if self.variant==None else self.variant)
        abstraction = [self.number_groups, self.coef_owners, self.coef_distribution_criterion, variant, self.selection_mode]
        for a_idx, c_idx1, c_idx2 in itertools.product(m.range_action_indices, self.range_class_indices, self.range_class_indices):
            step = step + one
            print(f"Abstraction {abstraction} ► Fetching the abstract transition and cost matrices ..... step {step.to(int).item()}/{num_steps.item()} ({(step/num_steps).item():.0%})", end="\r")
            transition_matrix[a_idx,c_idx1,c_idx2] = zero
            cost_matrix[a_idx,c_idx1] = zero
            class1, class2 = self.classes_list[c_idx1], self.classes_list[c_idx2]
            action = m.actions_matrix[a_idx]
            for s_idx1 in class1:
                state1 = m.states_matrix[s_idx1]
                probability = zero
                for s_idx2 in class2: 
                    state2 = m.states_matrix[s_idx2]
                    probability = probability + m.transition_probability_fn(state1, action, state2)
                weight = self.weight_distribution[s_idx1]
                transition_matrix[a_idx,c_idx1,c_idx2] = transition_matrix[a_idx,c_idx1,c_idx2] + weight*probability
                cost = m.cost_fn(state1,action)
                cost_matrix[a_idx,c_idx1] = cost_matrix[a_idx,c_idx1] + weight*cost
        self.transition_matrix, self.cost_matrix = transition_matrix, cost_matrix
        print(" "*150, end="\r")
        
    def resolution(self, discount_factor=None, precision=None) -> None:
        
        # Find the solution of the abstract MDP
        m = self.model
        discount_factor = (m.discount_factor if discount_factor==None else discount_factor)
        discount_factor = torch.tensor([discount_factor],dtype=float).to(device)
        error_factor = discount_factor / (one-discount_factor)
        precision = (m.precision if precision==None else precision)
        resolution_beginning_time = time.time()
        variant = ("--" if self.variant==None else self.variant)
        abstraction = [self.number_groups, self.coef_owners, self.coef_distribution_criterion, variant, self.selection_mode]
        num_states, num_actions = self.number_states, m.number_actions
        old_value, new_value = torch.zeros([num_states],dtype=float).to(device), torch.zeros([num_states],dtype=float).to(device)
        error = "#NA"
        policy = torch.zeros([num_states],dtype=int).to(device)
        while True:
            for c_idx in self.range_class_indices:  # Index of the current state. For each state, get in the new value function the optimal update of the old value function
                print(f"Abstraction {abstraction} ► Solving ..... Current error = {error};  current class number = {c_idx.item()}", end="\r")
                Q = torch.zeros([num_actions],dtype=float).to(device)  # Cost of applying each action
                for a in m.range_action_indices:
                    Q[a] = self.cost_matrix[a,c_idx] + discount_factor*torch.dot(self.transition_matrix[a,c_idx],old_value)
                new_value[c_idx] = torch.min(Q)  # Minimum possible cost in the current state
                policy[c_idx] = torch.argmin(Q)  # Action that causes this minimum cost
            differences_vector = new_value - old_value
            difference_max, difference_min = torch.max(differences_vector), torch.min(differences_vector)
            difference_scope = difference_max - difference_min
            error = (error_factor * difference_scope).item()  # Bound of the error of taking new_value as the optimal value function
            if error <= precision: break
            else: old_value = new_value.clone()
        print(" "*250, end="\r")
        self.abstract_solution = torch.stack([policy, new_value])
        resolution_end_time = time.time()
        self.resolution_elapse_time = resolution_end_time - resolution_beginning_time
        self.precision = error
        
        # Extrapolate the abstract solution to find the solution of the ground queuing problem
        extrapolation_beginning_time = time.time()
        solution = torch.zeros([2,m.number_states],dtype=float).to(device)
        for s_idx in m.range_state_indices:
            c_idx = self.states_vector[s_idx]
            solution[:,s_idx] = self.abstract_solution[:,c_idx]
        self.solution = solution
        extrapolation_end_time = time.time()
        self.extrapolation_elapse_time = extrapolation_end_time - extrapolation_beginning_time

In [47]:
class abstractions_comparison:
    
    def __init__(self, gm:ground_model) -> None:
        self.gm = gm
        self.get_numbers_groups_list()
        self.coefs_owners_list = ["UEs", "groups"]
        self.coefs_distribution_criteria_list = ["uniform", "sim", "dissim"]
        self.variants_list = ["sd", "cross", "gini"]
        self.selection_modes_list = ["one", "top", "all"]
        self.path = os.path.join(gm.directory, "Abstractions_comparison.csv")
        self.headers = ["ID", "n_groups", "coef_owners", "coef_criterion", "criterion_variant", "select_mode",
                        "max_diff_values", "avrg_diff_values", "max_diff_actions", "avrg_diff_actions",
                        "abstraction_time", "resolution_time", "extrapolation_time"]
    
    def get_numbers_groups_list(self) -> torch.tensor:  # Return the possible numbers of groups
        def number_groups_fn(N):
            model = self.gm
            characteristics_vector = model.number_UEs/model.number_RBs*torch.sum(model.CQI_matrix,dim=1) - model.arrival_rates_vector
            if N >= model.number_UEs:
                if characteristics_vector.unique().size(0) == characteristics_vector.size(0):
                    sorted_indices = torch.argsort(characteristics_vector)
                    num_elements = characteristics_vector.size(0)
                    vector_groups = torch.zeros(num_elements, dtype=torch.long)
                    vector_groups[sorted_indices] = torch.arange(num_elements)
                    return vector_groups
                else:
                    return number_groups_fn(model.number_UEs.item() - 1)
            quantiles = torch.linspace(0, 1, N+1)[1:-1].to(float).to(device)
            bounds_characteristics = torch.quantile(characteristics_vector, quantiles)
            _, group_indices_vector = torch.searchsorted(bounds_characteristics, characteristics_vector.unsqueeze(1), right=True).squeeze(1).unique(return_inverse=True)
            return group_indices_vector.unique().size(0)
        
        numbers_groups_list = []
        for target_number_groups in range(self.gm.number_UEs):
            number_groups = number_groups_fn(target_number_groups)
            if not number_groups in numbers_groups_list: numbers_groups_list.append(number_groups)
        self.numbers_groups_list = numbers_groups_list
        
    def launch(self):
        ID = 0
        total = 24*len(self.numbers_groups_list)
        gm = self.gm
        numbers_groups = self.numbers_groups_list
        owners = self.coefs_owners_list
        criteria = self.coefs_distribution_criteria_list
        variants = self.variants_list
        modes = self.selection_modes_list
        with open(self.path, mode="w", newline="") as destination:
            writer = csv.writer(destination)
            writer.writerow(self.headers)
            
            for ng,owner,criterion,mode in itertools.product(numbers_groups,owners,criteria,modes):
                if owner=="groups" or criterion=="dissim":
                    for variant in variants:
                        ID += 1
                        abstraction = ["groups="+str(ng), "owners="+str(owner), "criterion="+str(criterion), "variant="+str(variant), "selectionMode="+str(mode)]
                        print(f"Getting the performance of abstraction {abstraction}  =  abstraction {ID}/{total} ({ID/total:.0%})")
                        am = abstract_model(model=gm, number_groups=ng, coef_owners=owner, coef_distribution_criterion=criterion, variant=variant, selection_mode=mode)
                        am.resolution()
                        differences = torch.abs(gm.solution - am.solution)
                        max_diff_values = torch.max(differences[1]).item()
                        average_diff_values = torch.mean(differences[1]).item()
                        max_diff_actions = torch.max(differences[0]).to(int).item()
                        average_diff_actions = torch.mean(differences[0]).item()
                        row = [ID, ng, owner, criterion, variant, mode,
                               max_diff_values, average_diff_values, max_diff_actions, average_diff_actions,
                               am.abstraction_elapse_time, am.resolution_elapse_time, am.extrapolation_elapse_time]
                        writer.writerow(row)
                else:
                    ID += 1
                    abstraction = ["groups="+str(ng), "owners="+str(owner), "criterion="+str(criterion), "variant=--", "selectionMode="+str(mode)]
                    print(f"Getting the performance of abstraction {abstraction}  =  abstraction {ID}/{total} ({ID/total:.0%})")
                    am = abstract_model(model=gm, number_groups=ng, coef_owners=owner, coef_distribution_criterion=criterion, selection_mode=mode)
                    am.resolution()
                    differences = torch.abs(gm.solution - am.solution)
                    max_diff_values = torch.max(differences[1]).item()
                    average_diff_values = torch.mean(differences[1]).item()
                    max_diff_actions = torch.max(differences[0]).to(int).item()
                    average_diff_actions = torch.mean(differences[0]).item()
                    row = [ID, ng, owner, criterion, "--", mode,
                            max_diff_values, average_diff_values, max_diff_actions, average_diff_actions,
                            am.abstraction_elapse_time, am.resolution_elapse_time, am.extrapolation_elapse_time]
                    writer.writerow(row)
    
    def view(self):
        pass

## <h2 id=Tests>TESTS</h2>

### <h3>Ground Model for the Queuing Problem</h3>

In [6]:
m = ground_model(2,3,2)

Setting the arrival rate for each UE.
Arrival rate for UE 0: 
Arrival rate for UE 1: 
Arrival rate for UE 2: 


### <h3>Cost and Transition Functions</h3>

In [7]:
# TEST FOR TRANSITION PROBABILITIES

max_diff = "NA"
SN = "NA"
num_steps = (m.number_actions * m.number_states).item()
step = 0
num_states = m.number_states.item()
for ai in range(m.number_actions):
    AT = m.actions_matrix[ai]
    AN = AT.cpu().numpy()
    print(f"\n\nWith action a_{ai} = {AN}\n")
    for si1 in range(m.number_states):
        step += 1
        S1T = m.states_matrix[si1]
        S1N = S1T.cpu().numpy()
        S = torch.tensor([0]).to(device)
        state = 0
        for si2 in range(m.number_states):
            state += 1
            S2T = m.states_matrix[si2]
            PT = m.transition_probability_fn(S1T,AT,S2T)
            S = S + PT
            S2N, PN = S2T.cpu().numpy(), PT.item()
            # print(f"a_{ai}:      {S1N} --->  {S2N}:    P = {PN}")
            print(f"Step {step}/{num_steps} ({step/num_steps:.0%}):  Last sum of proba = {SN},  Max error = {max_diff}.     Progress in the current step: {state/num_states:.0%}                               ", end="\r")
        SN = S.item()
        # print(f"\n                  Sum of probabilities = {SN}\n\n")
        max_diff = (abs(SN-1) if max_diff == "NA" else max((max_diff, abs(SN-1))))
print(f"\nMax error = {max_diff}")



With action a_0 = [0 0]

Step 27/243 (11%):  Last sum of proba = 1.0,  Max error = 2.220446049250313e-16.     Progress in the current step: 100%                                             

With action a_1 = [0 1]

Step 54/243 (22%):  Last sum of proba = 0.9999999999999999,  Max error = 2.220446049250313e-16.     Progress in the current step: 100%                               

With action a_2 = [0 2]

Step 81/243 (33%):  Last sum of proba = 1.0,  Max error = 2.220446049250313e-16.     Progress in the current step: 100%                                              

With action a_3 = [1 0]

Step 108/243 (44%):  Last sum of proba = 0.9999999999999999,  Max error = 2.220446049250313e-16.     Progress in the current step: 100%                               

With action a_4 = [1 1]

Step 135/243 (56%):  Last sum of proba = 1.0,  Max error = 2.220446049250313e-16.     Progress in the current step: 100%                                              

With action a_5 = [1 2]

Step 162/243

In [50]:
# TEST OF COSTS

list_rests = list_costs = []
for a,s in product(m.range_action_indices,m.range_state_indices):
    action, state = m.actions_matrix[a], m.states_matrix[s]
    remainder = m.remainder_fn(state,action)
    cost = m.cost_fn(state, action)
    list_rests.append(torch.sum(remainder).item())
    list_costs.append(cost.item())
    print(f"Action {action.cpu().numpy()}  in state {state.cpu().numpy()}      --->      Rest = {remainder.cpu().numpy()}  and Cost = {cost.item()}")
print(f"\nThe correlation coefficient between the cost and the sum of the rest in UE buffers is {np.corrcoef(list_costs,list_rests)[0,1]}")

Action [0 0]  in state [0 0 0]      --->      Rest = [0 0 0]  and Cost = 0.0
Action [0 0]  in state [0 0 1]      --->      Rest = [0 0 1]  and Cost = 1.0
Action [0 0]  in state [0 0 2]      --->      Rest = [0 0 2]  and Cost = 1.9999999999999998
Action [0 0]  in state [0 1 0]      --->      Rest = [0 1 0]  and Cost = 0.9999999999999999
Action [0 0]  in state [0 1 1]      --->      Rest = [0 1 1]  and Cost = 2.0
Action [0 0]  in state [0 1 2]      --->      Rest = [0 1 2]  and Cost = 3.0
Action [0 0]  in state [0 2 0]      --->      Rest = [0 2 0]  and Cost = 2.0
Action [0 0]  in state [0 2 1]      --->      Rest = [0 2 1]  and Cost = 3.0
Action [0 0]  in state [0 2 2]      --->      Rest = [0 2 2]  and Cost = 4.0
Action [0 0]  in state [1 0 0]      --->      Rest = [0 0 0]  and Cost = 0.0
Action [0 0]  in state [1 0 1]      --->      Rest = [0 0 1]  and Cost = 1.0
Action [0 0]  in state [1 0 2]      --->      Rest = [0 0 2]  and Cost = 1.9999999999999998
Action [0 0]  in state [1 1 0] 

### <h3>Resolution</h3>

In [8]:
m.resolution()

In [52]:
for s in m.range_state_indices:
    action_number = m.solution[0,s].to(int).item()
    state = m.states_matrix[s].cpu().numpy()
    action = m.actions_matrix[action_number].cpu().numpy()
    print(f"State {state}  --->   {action}")

State [0 0 0]  --->   [0 0]
State [0 0 1]  --->   [0 2]
State [0 0 2]  --->   [2 2]
State [0 1 0]  --->   [0 1]
State [0 1 1]  --->   [1 2]
State [0 1 2]  --->   [2 2]
State [0 2 0]  --->   [1 1]
State [0 2 1]  --->   [1 2]
State [0 2 2]  --->   [2 2]
State [1 0 0]  --->   [0 0]
State [1 0 1]  --->   [0 2]
State [1 0 2]  --->   [2 2]
State [1 1 0]  --->   [0 1]
State [1 1 1]  --->   [1 2]
State [1 1 2]  --->   [2 2]
State [1 2 0]  --->   [1 1]
State [1 2 1]  --->   [1 2]
State [1 2 2]  --->   [2 2]
State [2 0 0]  --->   [0 0]
State [2 0 1]  --->   [0 2]
State [2 0 2]  --->   [2 2]
State [2 1 0]  --->   [0 0]
State [2 1 1]  --->   [0 2]
State [2 1 2]  --->   [0 2]
State [2 2 0]  --->   [0 0]
State [2 2 1]  --->   [0 2]
State [2 2 2]  --->   [0 0]


In [9]:
m.solution

tensor([[0.0000e+00, 2.0000e+00, 8.0000e+00, 1.0000e+00, 5.0000e+00, 8.0000e+00,
         4.0000e+00, 5.0000e+00, 5.0000e+00, 0.0000e+00, 2.0000e+00, 8.0000e+00,
         1.0000e+00, 5.0000e+00, 8.0000e+00, 4.0000e+00, 5.0000e+00, 5.0000e+00,
         0.0000e+00, 2.0000e+00, 8.0000e+00, 1.0000e+00, 2.0000e+00, 8.0000e+00,
         0.0000e+00, 2.0000e+00, 1.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00,
         7.2041e-03, 1.0036e+00, 2.0105e+00, 4.6246e-02, 4.6360e-02, 1.0001e+00,
         5.7396e-02, 1.0038e+00, 2.0017e+00, 1.0215e+00, 2.0143e+00, 3.0219e+00,
         3.0872e-01, 1.0923e+00, 2.0411e+00, 1.2669e+00, 2.1248e+00, 3.0909e+00,
         2.3989e+00, 3.3887e+00, 4.4870e+00]], device='cuda:0',
       dtype=torch.float64)

In [10]:
state = torch.tensor([3,0,0])
for a in m.range_action_indices:
    action = m.actions_matrix[a]
    cost = m.cost_fn(state,action).item()
    rest = m.remainder_fn(state,action).cpu().numpy()
    print(f"a_{a} = {action.cpu().numpy()} in {state.cpu().numpy()}: rest = {rest},  cost = {cost}")

a_0 = [0 0] in [3 0 0]: rest = [1 0 0],  cost = 1.0
a_1 = [0 1] in [3 0 0]: rest = [2 0 0],  cost = 1.9999999999999998
a_2 = [0 2] in [3 0 0]: rest = [2 0 0],  cost = 1.9999999999999998
a_3 = [1 0] in [3 0 0]: rest = [2 0 0],  cost = 1.9999999999999998
a_4 = [1 1] in [3 0 0]: rest = [3 0 0],  cost = 0.0
a_5 = [1 2] in [3 0 0]: rest = [3 0 0],  cost = 0.0
a_6 = [2 0] in [3 0 0]: rest = [2 0 0],  cost = 1.9999999999999998
a_7 = [2 1] in [3 0 0]: rest = [3 0 0],  cost = 0.0
a_8 = [2 2] in [3 0 0]: rest = [3 0 0],  cost = 0.0


<h3>Abstract Queuing Model (any)</h3>

In [11]:
am = abstract_model(m,2,"ue", "dis", "cross", "all")
# am.number_states

In [56]:
am.resolution(discount_factor=0.7, precision=1e-16)

In [57]:

print(am.abstraction_elapse_time)
print(am.resolution_elapse_time)
print(am.extrapolation_elapse_time)
print("____________________________")
print(am.abstraction_elapse_time + am.resolution_elapse_time + am.extrapolation_elapse_time)


177.45835423469543
1.1395964622497559
0.0009372234344482422
____________________________
178.59888792037964


In [58]:
for c_idx in range(len(am.classes_list)):
    states = am.classes_list[c_idx]
    abstract_state = am.states_matrix[c_idx].cpu().numpy()
    weight = am.weight_distribution[states].sum().item()
    print(f"Abstract state {abstract_state}:  total weight = {weight}")

Abstract state [0 0]:  total weight = 1.0
Abstract state [0 1]:  total weight = 1.0
Abstract state [0 2]:  total weight = 1.0
Abstract state [0 3]:  total weight = 1.0
Abstract state [0 4]:  total weight = 1.0
Abstract state [1 0]:  total weight = 1.0
Abstract state [1 1]:  total weight = 1.0
Abstract state [1 2]:  total weight = 1.0
Abstract state [1 3]:  total weight = 1.0
Abstract state [1 4]:  total weight = 1.0
Abstract state [2 0]:  total weight = 1.0
Abstract state [2 1]:  total weight = 1.0
Abstract state [2 2]:  total weight = 1.0
Abstract state [2 3]:  total weight = 1.0
Abstract state [2 4]:  total weight = 1.0


## <h2 id="Simulations">SIMULATIONS</h2>

<h3>Comparison of Abstract Models Performance</h3>

In [59]:
m = ground_model(2,3,2)
ac = abstractions_comparison(m)
ac.launch()

### <h3>Performance of the Best Abstract Model</h3>